In [10]:
import numpy as np

In [11]:
# Function to calculate the block size based on samples per mm and scanning velocity
def calculate_block_size(samples_per_mm, scanning_velocity_mm_s, sample_rate_hz):
    # Calculate how many samples are needed per second based on the scanning velocity and samples per millimeter
    samples_per_s = samples_per_mm * scanning_velocity_mm_s
    
    # Calculate block size based on the sample rate and the required samples per second
    block_size = sample_rate_hz / samples_per_s
    return block_size

# Constants
MCU_CLOCK_SPEED_HZ = 100e6  # Assume 100 MHz for Cortex-M4
OPERATIONS_PER_SAMPLE = 4    # Assume 2 adds, 1 multiply, 1 window function for the Goertzel algorithm
CHANNELS = 5                # Five channels running in parallel
SAFETY_FACTOR = 0.1         # 10% safety margin

# Input parameters
samples_per_mm = 15 / 2  # 15 samples per 2 mm
scanning_velocity_mm_s = 40  # Scanning velocity in mm/s
sample_rate_hz = 30e3  # Sample rate of the ADC in Hz (max 100 kHz)

# Calculate block size
block_size = calculate_block_size(samples_per_mm, scanning_velocity_mm_s, sample_rate_hz)

# Functions for calculation
def calculate_operations_per_block(block_size, ops_per_sample):
    return block_size * ops_per_sample

def calculate_execution_time_per_block(operations, clock_speed_hz):
    return operations / clock_speed_hz

def calculate_sample_period(sampling_rate_hz):
    return 1 / sampling_rate_hz

def calculate_total_execution_time(exec_time_per_block, channels):
    return exec_time_per_block * channels

def calculate_safety_margin(total_exec_time, safety_factor):
    return total_exec_time * (1 + safety_factor)

# Calculations
operations_per_block = calculate_operations_per_block(block_size, OPERATIONS_PER_SAMPLE)
execution_time_per_block = calculate_execution_time_per_block(operations_per_block, MCU_CLOCK_SPEED_HZ)
sample_period = calculate_sample_period(sample_rate_hz)
total_execution_time = calculate_total_execution_time(execution_time_per_block, CHANNELS)
total_execution_time_with_margin = calculate_safety_margin(total_execution_time, SAFETY_FACTOR)

In [12]:
# Results
print(f"Operations per block: {operations_per_block}")
print(f"Execution time per block (s): {execution_time_per_block:.10f}")
print(f"Sample period (s): {sample_period:.10f}")
print(f"Total execution time for {CHANNELS} channels (s): {total_execution_time:.10f}")
print(f"Total execution time with safety margin (s): {total_execution_time_with_margin:.10f}")
print(f"Can the MCU handle the load? {'YES' if sample_period > total_execution_time_with_margin else 'NO'}")

Operations per block: 400.0
Execution time per block (s): 0.0000040000
Sample period (s): 0.0000333333
Total execution time for 5 channels (s): 0.0000200000
Total execution time with safety margin (s): 0.0000220000
Can the MCU handle the load? YES
